In [ ]:
import torch
import zetta_utils
from zetta_utils.parsing import cue
from zetta_utils import builder

In [ ]:
print(torch.__version__)

In [ ]:
zetta_utils.load_all_modules()

In [ ]:
spec = '''
model: {
    "@type": "torch.nn.Sequential"
    modules: [
        {
            "@type": "torch.nn.Conv3d"
            in_channels: 1
            out_channels: 16
            kernel_size: [1, 5, 5]
            padding: [0, 2, 2]
            bias: false
        },
        {
            "@type": "torch.nn.InstanceNorm3d"
            num_features: 16
            affine: true
        },
        {
            "@type": "torch.nn.ReLU"            
            inplace: true
        },
        {
            "@type": "UNet"
            "@version": "0.0.1"
            conv: {
                "@type": "torch.nn.Conv3d"
                "@mode": "partial"
                bias: false
            }
            downsample: {
                "@type": "torch.nn.MaxPool3d"
                "@mode": "partial"
                kernel_size: [1, 2, 2]
            }
            upsample: {
                "@type": "UpConv"
                "@mode": "partial"
                kernel_size: [1, 1, 1]
                upsampler: {
                    "@type": "torch.nn.Upsample"
                    "@mode": "partial"
                    scale_factor: [1, 2, 2]
                    mode: "trilinear"                    
                }
                conv: {
                    "@type": "torch.nn.Conv3d"
                    "@mode": "partial"
                    bias: false
                }
            }
            activation: {
                "@type": "torch.nn.ReLU"
                "@mode": "partial"
                inplace: true
            }
            normalization: {
                "@type": "torch.nn.InstanceNorm3d"
                "@mode": "partial"
                affine: false
            }
            skips: {"1": 3}
            list_num_channels: [
                [16, 16, 16, 16, 16],
                [16, 32, 32, 32, 32],
                [32, 64, 64, 64, 64],

                [64, 128, 128, 128, 128],

                [64, 64, 64, 64, 64],
                [32, 32, 32, 32, 32],
                [16, 16, 16, 16, 16],
            ]
            kernel_sizes: [3, 3, 3]
            paddings: [1, 1 ,1]
            normalize_last: true
            activate_last: true
            activation_mode: "post"
        },
        {
            "@type": "torch.nn.Conv3d"
            in_channels: 16
            out_channels: 3
            kernel_size: [1, 5, 5]
            padding: [0, 2, 2]
            bias: true
        },
        {
            "@type": "CenterCrop"
            size: [16, 192, 192]
        },
        {
            "@type": "torch.nn.Sigmoid"
        },
    ]
}
'''

In [ ]:
result = builder.build(cue.loads(spec))

In [ ]:
model = result['model']

In [ ]:
x = torch.randn([1, 1, 20, 256, 256])
y = model(x)

In [ ]:
y.shape

In [ ]:
x = torch.randn([1, 1, 20, 256, 256])

torch.onnx.export(
    model,
    (x, {}),
    "ZRSUNet_post.onnx",
    verbose=False,
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
)